# 예제 2.1 토큰화 코드

In [1]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print(f"input_text_list : {input_text_list}")

input_text_list : ['나는', '최근', '파리', '여행을', '다녀왔다']


In [2]:
# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word : idx for idx, word in enumerate(input_text_list)}
idx2str = {idx : word for idx, word in enumerate(input_text_list)}
print(f"str2idx : {str2idx}")
print(f"idx2str : {idx2str}")

str2idx : {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str : {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}


In [3]:
# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print(f"input_ids : {input_ids}")

input_ids : [0, 1, 2, 3, 4]


# 예제 2.2 토큰 아이디에서 벡터로 변환

In [4]:
import torch.nn as nn
import torch

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

In [5]:
input_embeddings = embed_layer(torch.tensor(input_ids))
input_embeddings = input_embeddings.unsqueeze(0)
input_embeddings.shape

torch.Size([1, 5, 16])

# 예제 2.3 절대적 위치 인코딩

In [7]:
import torch.nn as nn
import torch

embedding_dim = 16
max_position = 12
position_embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids)) # (5,16)
token_embeddings = token_embeddings.unsqueeze(0) # (1,5,16)
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

# 2.4 쿼리, 키, 값 벡터를 만드는 nn.linear 층

In [9]:
import torch.nn as nn

head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
# (1,5,16)
querys = weight_q(input_embeddings)
print(f"querys : {querys}")

# (1,5,16)
keys = weight_k(input_embeddings)
print(f"keys : {keys}")

# (1,5,16)
values = weight_v(input_embeddings)
print(f"values : {values}")

querys : tensor([[[-0.2961,  0.6854,  0.1530, -0.3426, -0.0741,  0.3178,  0.8699,
          -0.3985, -0.5971, -0.6199, -0.5187, -0.6099,  0.0852, -0.2142,
          -0.0194,  0.4858],
         [ 0.4327,  1.1523, -0.8210, -0.5073,  0.9834, -0.1231,  0.0214,
          -0.0982, -0.3537, -0.5162,  1.0381, -0.9395, -1.3320,  0.8599,
           0.0905, -0.0640],
         [ 0.3872, -0.0376, -0.3188,  0.5390,  0.0762, -0.2741, -0.0638,
           0.1104,  0.5756, -0.1870,  1.6786, -1.5408,  0.1511, -0.9097,
          -0.9001, -1.6500],
         [-0.4406,  0.3879,  0.8989,  0.5641, -1.0762,  0.8097, -0.0108,
          -0.9281, -1.0690,  0.9965, -1.9264,  0.8267,  0.6689, -0.5009,
           0.1020,  0.0233],
         [-0.0690, -0.1553, -1.3841, -0.1809,  0.7826, -0.5997, -0.5168,
          -0.8792,  0.4327, -0.6208,  0.6876, -0.3782, -0.6043,  0.5955,
          -0.0898, -0.0042]]], grad_fn=<ViewBackward0>)
keys : tensor([[[ 0.5223,  0.1812,  1.3983, -0.6483,  0.8094, -0.6200, -1.0524,
         

# 2.5 스케일 점곱 방식의 어텐션

In [11]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal = False):
    dim_k = querys.size(-1) # 16
    # 1. 쿼리와 키를 곱한다. 분산이 커지는 것을 방지하기 위해 임베딩 차원 수의 제곱근으로 나눈다
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    
    # 2. 쿼리와 키를 곱해 게산한 스코어(scores)를 합이 1이 되도록 소프트맥스를 취해 가중치로 바꾼다
    weights = F.softmax(scores, dim=1)
    
    # 3. 가중치와 값을 곱해 입력과 동일한 형태의 출력을 반환한다
    return weights @ values

# 2.6 어텐션 연산의 입력과 출력

In [12]:
print(f"원본 입력 형태 : {input_embeddings.shape}")

after_attention_embeddings = compute_attention(querys, keys, values)

print(f"어텐션 적용 후 형태 : {after_attention_embeddings.shape}")

원본 입력 형태 : torch.Size([1, 5, 16])
어텐션 적용 후 형태 : torch.Size([1, 5, 16])


# 2.7 어텐션 연산의 입력과 출력

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        self.is_causal = is_causal
        # 쿼리 벡터 생성을 위한 선형 층
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        
        # 키 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        
        # 값 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim)
        
    def forward(self, querys, keys, values):
        outputs = compute_attention(
            # 쿼리 벡터
            self.weight_q(querys),
            
            # 키 벡터
            self.weight_k(keys),
            
            # 값 벡터
            self.weight_v(values),
            is_causal = self.is_causal
        )
    
        return outputs

# 2.8 멀티 헤드 어텐션 구현

In [ ]:
class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()
        self.n_head = n_head
        self.is_causal = is_causal
        # 쿼리 벡터 생성을 위한 선형 층
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        
        # 키 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        
        # 값 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim)
        
        self.concat_linear = nn.Linear(d_model, d_model)
        
    def forward(self, querys, keys, values):
        outputs = compute_attention(
            # 쿼리 벡터
            self.weight_q(querys),
            
            # 키 벡터
            self.weight_k(keys),
            
            # 값 벡터
            self.weight_v(values),
            is_causal = self.is_causal
        )
    
        return outputs